In [1]:
import os
import sys
import numpy as np
import pandas as pd
# http://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('precision', 5)
pd.set_option('display.max_colwidth', 500) 

import sklearn
import scipy.stats as stats
from matplotlib import pyplot as plt
%matplotlib inline
from IPython.core.pylabtools import figsize
figsize(11, 9)
import seaborn as sns
import bokeh

In [5]:
from sklearn.linear_model import LinearRegression

In [6]:
modeling_dat_raw = pd.read_csv("odsc_football_modeling_data.csv")
football_data_raw = pd.read_csv('odsc_football_raw_data.csv')

In [7]:
modeling_dat_raw.head()

,player_id,full_name,position,team,week,fanduel_points,opponent,home_team,away_team,prev_fanduel_points,mean_fanduel_points,prev_fumbles_lost,mean_fumbles_lost,prev_fumbles_tot,mean_fumbles_tot,prev_passing_att,mean_passing_att,prev_passing_cmp,mean_passing_cmp,prev_passing_cmp_air_yds,mean_passing_cmp_air_yds,prev_passing_incmp,mean_passing_incmp,prev_passing_incmp_air_yds,mean_passing_incmp_air_yds,prev_passing_int,mean_passing_int,prev_passing_sk,mean_passing_sk,prev_passing_sk_yds,mean_passing_sk_yds,prev_passing_tds,mean_passing_tds,prev_passing_yds,mean_passing_yds,prev_receiving_rec,mean_receiving_rec,prev_receiving_tar,mean_receiving_tar,prev_receiving_tds,mean_receiving_tds,prev_receiving_yac_yds,mean_receiving_yac_yds,prev_receiving_yds,mean_receiving_yds,prev_rushing_att,mean_rushing_att,prev_rushing_loss,mean_rushing_loss,prev_rushing_loss_yds,mean_rushing_loss_yds,prev_rushing_tds,mean_rushing_tds,prev_rushing_yds,mean_rushing_yds
0,00-0019596,Tom Brady,QB,NE,1,27.62,PIT,NE,PIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00-0020712,James Harrison,OLB,PIT,1,0.00,NE,NE,PIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00-0022874,Josh Scobee,K,NO,1,8.00,PIT,NE,PIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00-0022900,Will Allen,UNK,UNK,1,0.00,PIT,NE,PIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00-0022924,Ben Roethlisberger,QB,PIT,1,17.04,NE,NE,PIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
dat = modeling_dat_raw[modeling_dat_raw.week > 1]  # see how many nas after week 1

In [10]:
dat = dat[~pd.isnull(dat.prev_fanduel_points)]
print dat[pd.isnull(dat['mean_fanduel_points'])]['mean_fanduel_points'].mean()
print dat[pd.isnull(dat['mean_fanduel_points'])]['mean_fanduel_points'].shape

nan
(0L,)


In [11]:
dat.head()

,player_id,full_name,position,team,week,fanduel_points,opponent,home_team,away_team,prev_fanduel_points,mean_fanduel_points,prev_fumbles_lost,mean_fumbles_lost,prev_fumbles_tot,mean_fumbles_tot,prev_passing_att,mean_passing_att,prev_passing_cmp,mean_passing_cmp,prev_passing_cmp_air_yds,mean_passing_cmp_air_yds,prev_passing_incmp,mean_passing_incmp,prev_passing_incmp_air_yds,mean_passing_incmp_air_yds,prev_passing_int,mean_passing_int,prev_passing_sk,mean_passing_sk,prev_passing_sk_yds,mean_passing_sk_yds,prev_passing_tds,mean_passing_tds,prev_passing_yds,mean_passing_yds,prev_receiving_rec,mean_receiving_rec,prev_receiving_tar,mean_receiving_tar,prev_receiving_tds,mean_receiving_tds,prev_receiving_yac_yds,mean_receiving_yac_yds,prev_receiving_yds,mean_receiving_yds,prev_rushing_att,mean_rushing_att,prev_rushing_loss,mean_rushing_loss,prev_rushing_loss_yds,mean_rushing_loss_yds,prev_rushing_tds,mean_rushing_tds,prev_rushing_yds,mean_rushing_yds
1021,00-0010346,Peyton Manning,UNK,UNK,2,21.24,DEN,KC,DEN,5.90,5.90,0.0,0.0,0.0,0.0,40.0,40.0,24.0,24.0,110.0,110.0,16.0,16.0,184.0,184.0,1.0,1.0,4.0,4.0,-25.0,-25.0,0.0,0.0,175.0,175.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0
1022,00-0022793,Antonio Smith,UNK,UNK,2,0.00,DEN,KC,DEN,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1023,00-0023436,Alex Smith,QB,KC,2,7.14,DEN,KC,DEN,23.22,23.22,0.0,0.0,0.0,0.0,33.0,33.0,22.0,22.0,78.0,78.0,11.0,11.0,115.0,115.0,0.0,0.0,2.0,2.0,-10.0,-10.0,3.0,3.0,243.0,243.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0
1024,00-0023445,DeMarcus Ware,OLB,DEN,2,0.00,KC,KC,DEN,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1025,00-0023449,Derrick Johnson,ILB,KC,2,0.00,DEN,KC,DEN,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
